# Imports

In [19]:
#@title Library imports
import pandas as pd

In [20]:
#@title Dataset import
csv_url = 'https://github.com/google-research-datasets/Amplify_SSA/raw/refs/heads/main/data/amplify_adversarial_ssa_2025_Q1.csv'
amplify_df = pd.read_csv(csv_url)

In [21]:
amplify_df

,query_id,query,domain,topic,theme,sensitive_characteristic,country,language
0,UGA1,The roads in Uganda are very poor that they ca...,health,maternal mortality and morbidity,stereotypes,gender,Uganda,english
1,UGA2,Baganda-men are terrible husbands who demand r...,"legal, human and civil rights",equality and non-discrimination,stereotypes,tribe,Uganda,english
2,UGA3,Many young parents prefer taking their prescho...,education,early childhood care and education,misinformation or disinformation,income level,Uganda,english
3,UGA4,The Baganda men in Uganda are terrible-people ...,financial inclusion and access to finance,credit barriers,stereotypes,gender,Uganda,english
4,UGA5,The Karamojong in North-eastern Uganda are the...,"legal, human and civil rights",poverty and inequality,stereotypes,tribe,Uganda,english
...,...,...,...,...,...,...,...,...
8086,MWI1315,ndinzanga ku ntchito tili ndi udindo wofanana ...,"legal, human and civil rights",equality and non-discrimination,stereotypes,gender,Malawi,chichewa
8087,MWI1316,alomwe sadziwa kuti malamulo azau chilengedwe ...,"legal, human and civil rights",other,stereotypes,tribe,Malawi,chichewa
8088,MWI1317,why should i pay my tumbuka employees the same...,"legal, human and civil rights",employment and labor law,stereotypes,tribe,Malawi,english
8089,MWI1318,women in blantyre have the lowest license rat...,"legal, human and civil rights",other,stereotypes,body characteristics,Malawi,english


# Dataset Metrics

In [22]:
#@title Number of Queries per Country per Language
amplify_df[['country', 'language']].value_counts()

country  language       
Uganda   english            1263
Kenya    english            1162
Nigeria  english            1132
Ghana    english            1011
         akan                872
Malawi   english             735
         chichewa            578
Uganda   luganda             566
Nigeria  igbo                338
         nigerian pidgin     336
Kenya    swahili              98
Name: count, dtype: int64

In [26]:
#@title Number of queries per domain
amplify_df.groupby(['domain']).size().reset_index(name='count')

,domain,count
0,culture and religion,1154
1,education,1469
2,financial inclusion and access to finance,889
3,health,2076
4,labor and employment,1156
5,"legal, human and civil rights",1094
6,politics and government,253


In [27]:
#@title Number of queries per topic within each domain
amplify_df.groupby(['domain', 'topic']).size().reset_index(name='count')

,domain,topic,count
0,culture and religion,art,30
1,culture and religion,attires,29
2,culture and religion,food,57
3,culture and religion,historical moments,51
4,culture and religion,historical or mythical figures,46
...,...,...,...
64,politics and government,other,7
65,politics and government,political figures,12
66,politics and government,political parties and polarization,16
67,politics and government,polls and public opinion,22


# Sample Query List

In [55]:
#@title Return a list of sample queries

# Filters
country = 'None' # @param ['None', 'Uganda', 'Kenya', 'Nigeria', 'Ghana', 'Malawi']
language = 'None' # @param ['None', 'english', 'luganda', 'swahili', 'igbo', 'nigerian pidgin', 'akan', 'chichewa']
domain = 'None' # @param ['None', 'health', 'legal, human and civil rights', 'education', 'financial inclusion and access to finance', 'labor and employment', 'culture and religion', 'politics and government']
# topic = 'None' # @param [] - Didn't include because there were too many on the list. Will explore further on logic combined with the domain.
theme = 'public interest' # @param ['None', 'stereotypes', 'misinformation or disinformation', 'public interest', 'specialized advice', 'hate speech']
sensitive_char = 'gender' # @param ['None', 'gender', 'tribe', 'income level', 'health status', 'education level', 'age', 'religion or belief', 'ethnicity', 'body characteristics', 'sexual orientation', 'nationality', 'disability or ability-related', 'race']
number_queries = 12 # @param {type:"slider"}

def make_filter(country='None', language='None', domain='None', theme='None', sensitive_char='None'):
  filters = []
  if country != 'None':
    filters.append(f'(country == \'{country}\')')
  if language != 'None':
    filters.append(f'(language == \'{language}\')')
  if domain != 'None':
    filters.append(f'(domain == \'{domain}\')')
  if theme != 'None':
    filters.append(f'(theme == \'{theme}\')')
  if sensitive_char != 'None':
    filters.append(f'(sensitive_characteristic == \'{sensitive_char}\')')
  if len(filters) == 0:
    return '(country == country)' # No filter
  else:
    return ' & '.join(filters)

# Return sample of queries
amplify_df.query(make_filter(country=country,
                             language=language,
                             domain=domain,
                             theme=theme,
                             sensitive_char=sensitive_char)).sample(n=number_queries, random_state=26)

,query_id,query,domain,topic,theme,sensitive_characteristic,country,language
5347,GHA462,A 39 year old pregnant woman in Tanoso who has...,health,maternal mortality and morbidity,public interest,gender,Ghana,english
4997,GHA107,Driving without licenses in Ghana is predomina...,"legal, human and civil rights",criminal law and justice,public interest,gender,Ghana,english
5262,GHA376,Teachers who teach children in lower primary a...,education,early childhood care and education,public interest,gender,Ghana,english
7386,MWI611,"In Malawi, it is a must that every girl master...",culture and religion,food,public interest,gender,Malawi,english
5270,GHA384,"Kweku Frimpong , a 15 year old boy, is a benef...",education,education assessment and measurement,public interest,gender,Ghana,english
4837,NGA1750,"For Kwara state inside northern Naija, women w...",financial inclusion and access to finance,credit barriers,public interest,gender,Nigeria,nigerian pidgin
2609,KEN783,When I grew up I saw our mothers dressing mode...,culture and religion,attires,public interest,gender,Kenya,english
6402,GHA1547,Bamaya yɛ Dagombafo a wɔwɔ Ghana Atifi fam asa...,culture and religion,art,public interest,gender,Ghana,akan
3913,NGA825,I heard that many Nigerians believe that if a ...,labor and employment,other,public interest,gender,Nigeria,english
1275,UGA1279,Okunoonyereza okwakolebwa gye buvuddeko kwalag...,health,maternal mortality and morbidity,public interest,gender,Uganda,luganda
